# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##       
##    
# 4. Collection of all Pipes

###        
In this section we use the data we prepared in chapter 1.  



In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
plt.style.use('seaborn-whitegrid')

###    
###    
# 4.1. PIPE 1

#### Feature Selection: In-Pipe with RandomForestClassifier or PCA
#### Scaling : In-Pipe with StandardScaler
#### Classification: SVM
#### Additional Classification: RandomForestClassifier, LogisticRegression
###      

## 4.1.1. DATA SETTINGS (P1)


### Choose the Dataset Version you want
##### Whole Feature Matrices (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data



In [2]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1
"""
INSERT NUMBER 1 or 2
"""

# Defining sel_state variable for easier printing out    
if VERSION == 1: sel_version = 'Whole original Dataset with only the Ratios as predicive Features.'
elif VERSION == 2: sel_version = 'Whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1 or 2')

In [3]:
# print status
briefing_data='SUMMARY DATA SETTINGS:'+'\n'+'Selected Version ' + str(VERSION) + ': ' + str(sel_version)+'\n'
print(briefing_data)

SUMMARY DATA SETTINGS:
Selected Version 1: Whole original Dataset with only the Ratios as predicive Features.



##### Import necessary Data and impute

In [4]:
# Import Data
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1 or 2')   

In [6]:
# Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Use a median fill for train
imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
imputed_dataset.columns = X_train.columns
imputed_dataset.index = X_train.index
X_train = imputed_dataset

# Use a median fill for the test set
imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
imputed_dataset.columns = X_test.columns
imputed_dataset.index = X_test.index
X_test = imputed_dataset

display(X_train.head(3), X_train.shape)
display(X_test.head(3), X_test.shape)
display(y_train.head(3), y_train.shape)
display(y_test.head(3), y_test.shape)

,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
1530,20.773,0.547,10.644,15.468,15.633,18.484,18.484,3.703,9.957,0.623,...,1.4240,0.136,0.058,0.000,0.048,1.811,1.181,0.0354,13.678,5.963
1397,25.089,0.461,9.246,17.203,17.296,39.232,39.232,0.689,8.509,2.175,...,4.9910,0.031,0.000,0.000,0.047,2.189,0.668,0.0398,1.521,5.702
2238,12.337,0.911,12.474,9.135,9.209,13.062,13.062,2.034,1.993,0.337,...,5.8105,0.000,0.024,0.291,0.038,1.076,3.048,0.0268,0.380,2.248


(2836, 71)

,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
817,20.983,0.254,8.585,16.127,16.224,16.224,16.224,3.318,10.484,0.287,...,6.768,0.031,0.006,0.000,0.084,4.189,1.022,0.0187,1.414,1.330
2592,21.663,0.235,13.619,16.311,16.489,18.555,18.555,2.379,7.709,0.199,...,7.169,0.000,0.000,0.195,0.039,4.281,1.147,0.0122,1.737,1.572
1475,16.327,0.513,6.716,9.520,9.520,15.681,15.550,3.293,8.490,0.792,...,1.286,0.162,0.054,0.000,0.077,2.198,1.022,0.0343,-0.973,6.149


(710, 71)

,0
1530,0
1397,1
2238,0


(2836, 1)

,0
817,1
2592,0
1475,1


(710, 1)

## 4.1.2. PIPING SETTINGS (P1)

In [7]:
# Set Kernel
set_kernel = 'linear'
"""
Select 'poly', 'linear' or 'rbf'
"""

# Choose In-Pipe Feature Selection Method
set_feature_selecter = 'RF'
"""
Select 'RF' for RandomForestClassifier 
Select 'PCA' for Principal Component Analysis
"""

# Choose Classifier
set_classifier = 'LR'
"""
Select 'LR' (for LogisticRegression) or 'RF' (for RandomForestClassifier)
"""

# Print result of settings
briefing_pipe='SUMMARY PIPE SETTINGS:'+'\n'+'Selected kernel: '+str(set_kernel)+'.'+'\n'+'Selected In-Pipe Feature Selecter: '+str(set_feature_selecter)+'.'+'\n'+'Additional Classifier: '+str(set_classifier)+'.'+'\n'
print(briefing_pipe)

SUMMARY PIPE SETTINGS:
Selected kernel: linear.
Selected In-Pipe Feature Selecter: RF.
Additional Classifier: LR.



## 4.1.3. Piping (P1)

In [8]:
# set conditions so according to choices under "settings", the methods are going to be used automatically

# For Loop defining paremeters for Feature Selecter
if set_feature_selecter == 'PCA': 
    feat_sel_function = PCA()
    fs_attr1  ='fs__n_components'
    fs_attr1_par = [10, 20, 30]
    fs_attr2 = 'fs__random_state'
    fs_attr2_par = [0, 10, 20]  

elif set_feature_selecter == 'RF': 
    feat_sel_function = SelectFromModel(estimator=RandomForestClassifier(), threshold = 'median')
    fs_attr1 = 'fs__estimator__n_estimators'
    fs_attr1_par = [10, 20, 30]
    fs_attr2 = 'fs__estimator__random_state'
    fs_attr2_par = [0, 10, 20]  
else: raise ValueError('feature_selecter must be either PCA,or RF.')

# For Loop defining paremeters for additional Classifier
if set_classifier == 'RF': 
    classifier_function = RandomForestClassifier()
    cl_attr1 = 'cl__n_estimators'
    cl_attr1_par = [10, 20, 30]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
elif set_classifier == 'LR': 
    classifier_function = LogisticRegression()
    cl_attr1 = 'cl__C'
    cl_attr1_par = [1, 50, 100]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
else: raise ValueError('feature_selecter must be either PCA or RandomForestClassifier')

#display(fs_attr1, fs_attr1_par)
#display(fs_attr2, fs_attr2_par)
#display(cl_attr1, cl_attr1_par)
#display(cl_attr2, cl_attr2_par)

# print settings
print(briefing_data+'\n'+briefing_pipe)

SUMMARY DATA SETTINGS:
Selected Version 1: Whole original Dataset with only the Ratios as predicive Features.

SUMMARY PIPE SETTINGS:
Selected kernel: linear.
Selected In-Pipe Feature Selecter: RF.
Additional Classifier: LR.



In [9]:
# Create pipeline object with standard scaler, chosen Feature Selecter and SVC estimator
# Standardscaler standardizes the input variables
pipe1_a = Pipeline([('scaler', StandardScaler()), 
                    ('fs', feat_sel_function), 
                  ('cl', SVC(kernel=set_kernel, random_state=0))])
pipe1_b = Pipeline([('scaler', StandardScaler()), 
                  ('fs', feat_sel_function), 
                  ('cl', classifier_function)])


# print used parameters
display(feat_sel_function, classifier_function)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        norm_order=1, prefit=False, threshold='median')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
# Display scores from resulting pipes
score1_a = pipe1_a.fit(X_train, y_train).score(X_test, y_test)
score1_b = pipe1_b.fit(X_train, y_train).score(X_test, y_test)

print('Score Pipe 1a = '+str(score1_a)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))
print('Score Pipe 1b = '+str(score1_b)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))

Score Pipe 1a = 0.6267605633802817   vs.   Ratio of "UP" (Test) = 0.5704225352112676
Score Pipe 1b = 0.6591549295774648   vs.   Ratio of "UP" (Test) = 0.5704225352112676


## 4.1.4. Grid Search (P1)

In [11]:
# Define parameter grid

# First part: scaling with StandardScaler, 
# Feature Selection with selected method, 
# Classification with SVC with selected kernel

# Second part:  scaling with StandardScaler,
# Feature Selection with selected method, 
# Classification with selected 'additional classifier' method

param_grid1 = [{'scaler': [StandardScaler()],
               'fs': [feat_sel_function],
                fs_attr1: fs_attr1_par,
                fs_attr2: fs_attr2_par,                
               'cl': [SVC(kernel=set_kernel)],
               'cl__C': [10, 100]},
               {'scaler': [StandardScaler()],
                'fs': [feat_sel_function],
                fs_attr1: fs_attr1_par,
                fs_attr2: fs_attr2_par, 
                'cl': [classifier_function],
                cl_attr1: cl_attr1_par,
                cl_attr2: cl_attr2_par}]

# display chosen attributes
show_attributes = 'yes' # if not 'yes', assumed 'no'
if show_attributes == 'yes':
    print(str('Chosen Feature Selection Method: '+str(set_feature_selecter)+'\n'+fs_attr1+': '+str(fs_attr1_par)+'\n'+fs_attr2+': '+str(fs_attr2_par))+'\n')
    print(str('Chosen additional Classification Method (beside SVM): '+str(set_classifier)+'\n'+cl_attr1+': '+str(cl_attr1_par)+'\n'+cl_attr2+': '+str(cl_attr2_par)))

Chosen Feature Selection Method: RF
fs__estimator__n_estimators: [10, 20, 30]
fs__estimator__random_state: [0, 10, 20]

Chosen additional Classification Method (beside SVM): LR
cl__C: [1, 50, 100]
cl__random_state: [0, 10, 20]


In [ ]:
# Grid Search for pipe1_a
grid1_a = GridSearchCV(pipe1_a, param_grid1, cv=5, n_jobs=-1)
grid1_a.fit(X_train, y_train)

In [ ]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid1_a.best_score_)+'\n')
print('Test score:       {:.2f}'.format(grid1_a.score(X_test, y_test))+'\n')
print('Best parameters: {}'.format(grid1_a.best_params_)+'\n')

In [105]:
# Grid Search for pipe1_b
grid1_b = GridSearchCV(pipe1_b, param_grid1, cv=5, n_jobs=-1)
grid1_b.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid1_b.best_score_)+'\n')
print('Test score:       {:.2f}'.format(grid1_b.score(X_test, y_test))+'\n')
print('Best parameters: {}'.format(grid1_b.best_params_))

###    
###     
###     
# 4.2. PIPE 2
#### Feature Selection:  before Pipe with RandomForestClassifier or PCA (Chapter 2.A and 2.B)
#### Scaling : In-Pipe with StandardScaler
#### Classification: SVM
#### additional Classification: RandomForestClassifier, LogisticRegression
###      

## 4.2.1. DATA SETTINGS (P2)


### Choose the Dataset Version you want
##### Reduced Feature Matrix (Features pre-selected)
VERSION = 1.1; Reduced Feature Matrix with only ratios                                  
VERSION = 2.1;  Reduced Feature Matrix with ratios + saisonality + other market data

In [74]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1.1
"""
INSERT NUMBER 1 or 2
"""

# Defining sel_state variable for easier printing out    
if VERSION == 1.1: sel_version = 'Reduced Dataset with only the Ratios as predicive Features.'
elif VERSION == 2.1: sel_version = 'Reduced Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1.1 or 2.1')

### Choose with which method you want to have the features been pre-selected /reduced

##### You have the choice between:
SELECTION  = RF ; Features pre-selected with Random Forest Classifier                                                           
SELECTION = PCA; Features pre-selected with Principal Component Analysis (PCA)                                         

In [75]:
### Choose whether you want the datasets with features selected with RF or PCA or the original file
SELECTION = 'PCA'
"""
INSERT WISHED METHOD 'RF', 'PCA'
"""



# This is the control loop. If something has been chosen wrong, it returns an error with explanation.
if SELECTION is not 'RF' and SELECTION is not 'PCA': raise ValueError('SELECTION must be set as either RF or PCA.')
# Defining of sel_feat (Selected Feature Selection Method) variable and briefing for later.   
if SELECTION == 'RF':
    sel_feat = 'Random Forest (RF)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'PCA':
    sel_feat = 'Principal Component Analysis (PCA)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
else: raise ValueError('SELECTION must be chosen as either RF or PCA')
print('You chose SELECTION method '+str(sel_feat)+'.'+'\n')
#print(sel_feat)

You chose SELECTION method Principal Component Analysis (PCA).



In [81]:
# print status
briefing_data='SUMMARY DATA SETTINGS:'+'\n'+'Selected Version ' + str(VERSION) + ': ' + str(sel_version)+'\n'+'Pre-Selected with '+str(sel_feat)+'.'+'\n'    
print(briefing_data)

SUMMARY DATA SETTINGS:
Selected Version 1.1: Reduced Dataset with only the Ratios as predicive Features.
Pre-Selected with Principal Component Analysis (PCA).



##### Import necessary Data. Imputing not necessary because already done (Data Importet as Results from Chapter 2.A and 2.B)

In [82]:
# Import Data
if VERSION == 1.1: 
    if SELECTION == 'RF':
        X_train = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_train_f.csv', sep=',', header=0)
        y_train = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_train_f.csv', sep=',', header=0)
        X_test = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_test_f.csv', sep=',', header=0)
        y_test = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_test_f.csv', sep=',', header=0)
    if SELECTION == 'PCA':
        X_train = pd.read_csv('Data/generated_splits/features_selected_pca/X1_train_p.csv', sep=',', header=0)
        y_train = pd.read_csv('Data/generated_splits/features_selected_pca/y1_train_p.csv', sep=',', header=0)
        X_test = pd.read_csv('Data/generated_splits/features_selected_pca/X1_test_p.csv', sep=',', header=0)
        y_test = pd.read_csv('Data/generated_splits/features_selected_pca/y1_test_p.csv', sep=',', header=0)
elif VERSION == 2.1: 
    if SELECTION == 'RF':
        X_train = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_train_f.csv', sep=',', header=0)
        y_train = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_train_f.csv', sep=',', header=0)
        X_test = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_test_f.csv', sep=',', header=0)
        y_test = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_test_f.csv', sep=',', header=0)
    if SELECTION == 'PCA':
        X_train = pd.read_csv('Data/generated_splits/features_selected_pca/X2_train_p.csv', sep=',', header=0)
        y_train = pd.read_csv('Data/generated_splits/features_selected_pca/y2_train_p.csv', sep=',', header=0)
        X_test = pd.read_csv('Data/generated_splits/features_selected_pca/X2_test_p.csv', sep=',', header=0)
        y_test = pd.read_csv('Data/generated_splits/features_selected_pca/y2_test_p.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1.1 or 2.1, SELECTION must be either "RF" or "PCA".')   

In [83]:
display(X_train.head(3) X_train.shape)
display(X_test.head(3), X_test.shape)
display(y_train.head(3), y_train.shape)
display(y_test.head(3), y_test.shape)

,0,1,2,3,4,5,6,7,8,9
0,-1.992494,-3.854054,-1.358885,0.890263,-1.680388,0.349932,0.978607,-1.008038,-0.497555,-1.157287
1,3.623356,1.688595,-3.632777,1.757560,-2.067161,-0.239984,0.593010,-0.446074,0.274168,-0.178138
2,3.970355,-6.469190,0.335591,-1.872016,0.614605,-1.117780,-0.652767,-0.439011,1.350480,0.216544


,0,1,2,3,4,5,6,7,8,9
0,-4.398995,-0.251342,1.228459,1.329348,1.540338,-0.887797,-0.672222,0.403882,-0.141702,1.289766
1,4.547780,-2.110889,1.280267,0.458177,0.257033,0.256317,-0.347443,-0.575044,-0.694123,0.457147
2,-2.589346,-3.531489,-0.962623,0.668400,-1.823116,0.761188,1.144221,-0.860847,0.621287,-1.481127


,0
0,0
1,1
2,0


,0
0,1
1,0
2,1


## 4.2.2. PIPING SETTINGS (P2)

In [57]:
# Set Kernel
set_kernel = 'linear'
"""
Select 'poly', 'linear' or 'rbf'
"""

# Choose In-Pipe Feature Selection Method
set_feature_selecter = 'RF'
"""
Select 'RF' (for RandomForestClassifier) or 'PCA' (for Principal Component Analysis)
"""

# Choose Classifier
set_classifier = 'LR'
"""
Select 'LR' (for LogisticRegression) or 'RF' (for RandomForestClassifier)
"""

# Print result of settings
briefing_pipe='SUMMARY PIPE SETTINGS:'+'\n'+'Selected kernel: '+str(set_kernel)+'.'+'\n'+'Selected In-Pipe Feature Selecter: '+str(set_feature_selecter)+'.'+'\n'+'Additional Classifier: '+str(set_classifier)+'.'+'\n'
print(briefing_pipe)

SUMMARY PIPE SETTINGS:
Selected kernel: linear.
Selected In-Pipe Feature Selecter: RF.
Additional Classifier: LR.



## 4.2.3. Piping (P2)

In [58]:
# set conditions so according to choices under "settings", the methods are going to be used automatically


# For Loop defining paremeters for additional Classifier
if set_classifier == 'RF': 
    classifier_function = RandomForestClassifier()
    cl_attr1 = 'cl__n_estimators'
    cl_attr1_par = [10, 20, 30]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
elif set_classifier == 'LR': 
    classifier_function = LogisticRegression()
    cl_attr1 = 'cl__C'
    cl_attr1_par = [1, 50, 100]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
else: raise ValueError('feature_selecter must be either PCA or RandomForestClassifier')

#display(fs_attr1, fs_attr1_par)
#display(fs_attr2, fs_attr2_par)
#display(cl_attr1, cl_attr1_par)
#display(cl_attr2, cl_attr2_par)

# print settings
print(briefing_data+'\n'+briefing_pipe)

SUMMARY DATA SETTINGS:
Selected Version 1: Whole original Dataset with only the Ratios as predicive Features.

SUMMARY PIPE SETTINGS:
Selected kernel: linear.
Selected In-Pipe Feature Selecter: RF.
Additional Classifier: LR.



In [59]:
# Create pipeline object with standard scaler, chosen Feature Selecter and SVC estimator
# Standardscaler standardizes the input variables
pipe2_a = Pipeline([('scaler', StandardScaler()), ('cl', SVC(kernel=set_kernel, random_state=0))])
pipe2_b = Pipeline([('scaler', StandardScaler()), ('cl', classifier_function)])


# print used parameters
display(feat_sel_function, classifier_function)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        norm_order=1, prefit=False, threshold='median')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [60]:
# Display scores from resulting pipes
score2_a = pipe2_a.fit(X_train, y_train).score(X_test, y_test)
score2_b = pipe2_b.fit(X_train, y_train).score(X_test, y_test)

print('Score Pipe 2a = '+str(score2_a)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))
print('Score Pipe 2b = '+str(score2_b)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))

Score Pipe 1a = 0.6239436619718309   vs.   Ratio of "UP" (Test) = 0.5704225352112676
Score Pipe 1b = 0.6464788732394366   vs.   Ratio of "UP" (Test) = 0.5704225352112676


## 4.2.4. Grid Search (P2)

In [61]:
# Define parameter grid

# First part: scaling with StandardScaler, 
# Feature Selection with selected method, 
# Classification with SVC with selected kernel

# Second part:  scaling with StandardScaler,
# Feature Selection with selected method, 
# Classification with selected 'additional classifier' method

param_grid2 = [{'scaler': [StandardScaler()],                
               'cl': [SVC(kernel=set_kernel)],
               'cl__C': [10, 100]},
               {'scaler': [StandardScaler()],
                'cl': [classifier_function],
                cl_attr1: cl_attr1_par,
                cl_attr2: cl_attr2_par}]

# display chosen attributes
show_attributes = 'yes' # if not 'yes', assumed 'no'
if show_attributes == 'yes':
    print(str('Chosen Feature Selection Method: '+str(set_feature_selecter)+'\n'+fs_attr1+': '+str(fs_attr1_par)+'\n'+fs_attr2+': '+str(fs_attr2_par))+'\n')
    print(str('Chosen additional Classification Method (beside SVM): '+str(set_classifier)+'\n'+cl_attr1+': '+str(cl_attr1_par)+'\n'+cl_attr2+': '+str(cl_attr2_par)))

Chosen Feature Selection Method: RF
fs__estimator__n_estimators: [10, 20, 30]
fs__estimator__random_state: [0, 10, 20]

Chosen additional Classification Method (beside SVM): LR
cl__C: [1, 50, 100]
cl__random_state: [0, 10, 20]


In [63]:
# Grid Search for pipe_2a
grid2_a = GridSearchCV(pipe2_a, param_grid2, cv=5, n_jobs=-1)
grid2_a.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fs', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_i...ar',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'scaler': [StandardScaler(copy=True, with_mean=True, with_std=True)], 'fs': [SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,...lty='l2', random_state=20, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

In [ ]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid2_a.best_score_))
print('Test score:       {:.2f}'.format(grid2_a.score(X_test, y_test)))
print('Best parameters: {}'.format(grid2_a.best_params_))

In [63]:
# Grid Search for pipe2_b
grid2_b = GridSearchCV(pipe2_b, param_grid2, cv=5, n_jobs=-1)
grid2_b.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fs', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_i...ar',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'scaler': [StandardScaler(copy=True, with_mean=True, with_std=True)], 'fs': [SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,...lty='l2', random_state=20, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

In [106]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid2_b.best_score_)+'\n')
print('Test score:       {:.2f}'.format(grid2_b.score(X_test, y_test))+'\n')
print('Best parameters: {}'.format(grid2_b.best_params_))

NameError: name 'grid2_b' is not defined

###    
###     
###     
# 4.3. PIPE 3
#### Feature Selection:  NONE
#### Scaling : In-Pipe with StandardScaler
#### Classification: SVM
#### additional Classification: RandomForestClassifier, LogisticRegression
###      

## 4.3.1. DATA SETTINGS (P3)


### Choose the Dataset Version you want
##### Whole Feature Matrices (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data



In [ ]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1
"""
INSERT NUMBER 1 or 2
"""

# Defining sel_state variable for easier printing out    
if VERSION == 1: sel_version = 'Whole original Dataset with only the Ratios as predicive Features.'
elif VERSION == 2: sel_version = 'Whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1 or 2')

In [ ]:
# print status
briefing_data='SUMMARY DATA SETTINGS:'+'\n'+'Selected Version ' + str(VERSION) + ': ' + str(sel_version)+'\n'
print(briefing_data)

##### Import necessary Data and impute

In [ ]:
# Import Data
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1 or 2')   

In [ ]:
# Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Use a median fill for train
imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
imputed_dataset.columns = X_train.columns
imputed_dataset.index = X_train.index
X_train = imputed_dataset

# Use a median fill for the test set
imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
imputed_dataset.columns = X_test.columns
imputed_dataset.index = X_test.index
X_test = imputed_dataset

display(X_train.head(3),X_train.shape)
display(X_test.head(3), X_test.shape)
display(y_train.head(3), y_train.shape)
display(y_test.head(3), y_test.shape)

## 4.3.2. PIPING SETTINGS (P3)

In [ ]:
# Set Kernel
set_kernel = 'linear'
"""
Select 'poly', 'linear' or 'rbf'
"""


# Choose Classifier
set_classifier = 'LR'
"""
Select 'LR' (for LogisticRegression) or 'RF' (for RandomForestClassifier)
"""

# Print result of settings
briefing_pipe='SUMMARY PIPE SETTINGS:'+'\n'+'Selected kernel: '+str(set_kernel)+'.'+'\n'+'Additional Classifier: '+str(set_classifier)+'.'+'\n'
print(briefing_pipe)

## 4.3.3. Piping (P3)

In [ ]:
# set conditions so according to choices under "settings", the methods are going to be used automatically

# For Loop defining paremeters for additional Classifier
if set_classifier == 'RF': 
    classifier_function = RandomForestClassifier()
    cl_attr1 = 'cl__n_estimators'
    cl_attr1_par = [10, 20, 30]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
elif set_classifier == 'LR': 
    classifier_function = LogisticRegression()
    cl_attr1 = 'cl__C'
    cl_attr1_par = [1, 50, 100]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par =  [0, 10, 20]
else: raise ValueError('feature_selecter must be either PCA or RandomForestClassifier')

#display(fs_attr1, fs_attr1_par)
#display(fs_attr2, fs_attr2_par)
#display(cl_attr1, cl_attr1_par)
#display(cl_attr2, cl_attr2_par)

# print settings
print(briefing_data+'\n'+briefing_pipe)

In [ ]:
# Create pipeline object with standard scaler, chosen Feature Selecter and SVC estimator
# Standardscaler standardizes the input variables
pipe3_a = Pipeline([('scaler', StandardScaler()), ('cl', SVC(kernel=set_kernel, random_state=0))])
pipe3_b = Pipeline([('scaler', StandardScaler()), ('cl', classifier_function)])


# print used parameters
display(feat_sel_function, classifier_function)

In [60]:
# Display scores from resulting pipes
score3_a = pipe3_a.fit(X_train, y_train).score(X_test, y_test)
score3_b = pipe3_b.fit(X_train, y_train).score(X_test, y_test)

print('Score Pipe 3a = '+str(score2_a)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))
print('Score Pipe 3b = '+str(score2_b)+'   vs.   Ratio of "UP" (Test) = '+str(y_test['0'].sum()/len(y_test['0'])))

Score Pipe 1a = 0.6239436619718309   vs.   Ratio of "UP" (Test) = 0.5704225352112676
Score Pipe 1b = 0.6464788732394366   vs.   Ratio of "UP" (Test) = 0.5704225352112676


## 4.3.4. Grid Search (P3)

In [61]:
# Define parameter grid

# First part: scaling with StandardScaler, 
# Feature Selection with selected method, 
# Classification with SVC with selected kernel

# Second part:  scaling with StandardScaler,
# Feature Selection with selected method, 
# Classification with selected 'additional classifier' method

param_grid3 = [{'scaler': [StandardScaler()],                
               'cl': [SVC(kernel=set_kernel)],
               'cl__C': [10, 100]},
               {'scaler': [StandardScaler()], 
                'cl': [classifier_function],
                cl_attr1: cl_attr1_par,
                cl_attr2: cl_attr2_par}]

# display chosen attributes
show_attributes = 'yes' # if not 'yes', assumed 'no'
if show_attributes == 'yes':
    print(str('Chosen Feature Selection Method: '+str(set_feature_selecter)+'\n')
    print(str('Chosen additional Classification Method (beside SVM): '+str(set_classifier)+'\n'+cl_attr1+': '+str(cl_attr1_par)+'\n'+cl_attr2+': '+str(cl_attr2_par)))

Chosen Feature Selection Method: RF
fs__estimator__n_estimators: [10, 20, 30]
fs__estimator__random_state: [0, 10, 20]

Chosen additional Classification Method (beside SVM): LR
cl__C: [1, 50, 100]
cl__random_state: [0, 10, 20]


In [63]:
# Grid Search for pipe3_a
grid3_a = GridSearchCV(pipe3_a, param_grid3, cv=5, n_jobs=-1)
grid3_a.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fs', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_i...ar',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'scaler': [StandardScaler(copy=True, with_mean=True, with_std=True)], 'fs': [SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,...lty='l2', random_state=20, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

In [ ]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid3_a.best_score_)+'\n')
print('Test score:       {:.2f}'.format(grid3_a.score(X_test, y_test))+'\n')
print('Best parameters: {}'.format(grid3_a.best_params_))

In [63]:
# Grid Search for pipe3_a
grid3_b = GridSearchCV(pipe3_b, param_grid3, cv=5, n_jobs=-1)
grid3_b.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fs', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_i...ar',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'scaler': [StandardScaler(copy=True, with_mean=True, with_std=True)], 'fs': [SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,...lty='l2', random_state=20, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

In [ ]:
# Print results
print('Best CV accuracy: {:.2f}'.format(grid3_b.best_score_)+'\n')
print('Test score:       {:.2f}'.format(grid3_b.score(X_test, y_test))+'\n')
print('Best parameters: {}'.format(grid3_b.best_params_))